First we load the review text...

In [2]:
data = open('./review_text.txt', 'r').read()
chars = list(set(data))
VOCAB_SIZE = len(chars)

Next we create two dictionaries for mapping the examples to features and vice-versa:

In [3]:
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

In [4]:
import numpy as np

seq_length = 50
X = np.zeros((len(data)/seq_length, seq_length, VOCAB_SIZE))
y = np.zeros((len(data)/seq_length, seq_length, VOCAB_SIZE))
for i in range(0, len(data)/seq_length):
    X_sequence = data[i*seq_length:(i+1)*seq_length]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((seq_length, VOCAB_SIZE))
    for j in range(seq_length):
        input_sequence[j][X_sequence_ix[j]] = 1.
        X[i] = input_sequence
        
    y_sequence = data[i*seq_length+1:(i+1)*seq_length+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((seq_length, VOCAB_SIZE))
    for j in range(seq_length):
        target_sequence[j][y_sequence_ix[j]] = 1.
        y[i] = target_sequence
       


Now it's learnin' time :-)

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed

model = Sequential()
hidden_dim = 500
layer_num = 2

model.add(LSTM(hidden_dim, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(layer_num - 1):
    model.add(LSTM(hidden_dim, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

Using TensorFlow backend.


Before we go further, a function for generating text given the model

In [8]:
def generate_text(model, length, vocab_size, ix_to_char):
    # start with a random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print ix_to_char[ix[-1]],
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

Let's see how bad our untrained model is

In [9]:
generate_length = 500

print(generate_text(model, generate_length, VOCAB_SIZE, ix_to_char))

S c d d b d b n n D o o o . . m a a a a W W W W W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W W W W W b W W W W W W W W 

OK, that's really bad. Now we train...

In [ ]:
nb_epoch = 0
batch_size = 50

while True:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    model.fit(X, y, batch_size=batch_size, verbose=1, nb_epoch=1)
    nb_epoch += 1
    generate_text(model, generate_length, VOCAB_SIZE, ix_to_char)
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(layer_num, hidden_dim, nb_epoch))



Epoch: 0



/usr/local/lib/python2.7/dist-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
5/5 [==============================] - 0s - loss: 3.5845
p                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    